这个章节官方文档可以参考[这里](https://python.langchain.com/docs/concepts/structured_outputs/)，准确来说是关于 `Structured outputs` 而并不局限于 JSON Parser 这一类工具。在这一部分我们会指定特殊的 prompt，让 LLM 能够返回给我们特定结构的数据。

**为什么重要呢？** 一般的 LLM 返回语句都是贴近人的自然语言，而非可以结构化存储的文件（比如 JSON、XML 等等）。因此在常见的 LLM 应用当中，我们可以指定让 LLM 返回结构化的数据，从而减少人为的数据筛选和清洗，最大化利用 LLM 的优势。

在官方文档中这张图表述了结构化输出的流程：
![Returning structured output](https://python.langchain.com/assets/images/structured_output-2c42953cee807dedd6e96f3e1db17f69.png)

关键点在于两个：
1. 我们可以通过一些定义方式将输出结构表示为 `Schema`；
2. 给定 `Schema` 给 LLM，让其输出为规定的结构化语言数据。

In [1]:
# 定义我们各项前置变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,  # 让我们的机器人变得严谨且准确
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

有一个 Python 库可以帮助我们进行类型提示和验证 `pydantic`。

In [2]:
!pip install pydantic

In [3]:
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")

在这里我们可以提前引入 tools 的用法，我们将刚刚定义的 `ResponseFormatter` 与我们的 LLM 绑定在一起，从而让 LLM 有了结构化输出的能力。这里是 LangChain 兼容性强的一点，能够使用原生接口与 pydantic 进行绑定。

不过准确来说是 `ChatOpenAI` 这个类自带的一种兼容能力。可以参考[官方文档](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)的 Structure Output 这一栏。

In [4]:
llm_with_tools = llm.bind_tools([ResponseFormatter])
ai_msg = llm_with_tools.invoke("What is Low-Rank Decomposition?")

In [9]:
print(ai_msg.tool_calls[0]["args"])

{'answer': 'Low-rank decomposition is a mathematical technique used to approximate a matrix by a product of two or more matrices with lower ranks. This method is particularly useful in data compression, noise reduction, and feature extraction. The idea is to represent a large matrix with a smaller set of data, capturing the most significant features while discarding less important information. Common methods for low-rank decomposition include Singular Value Decomposition (SVD), Principal Component Analysis (PCA), and Non-negative Matrix Factorization (NMF). These techniques are widely used in fields such as machine learning, statistics, and signal processing.', 'followup_question': 'How is Singular Value Decomposition (SVD) related to low-rank decomposition?'}


In [ ]:
# 可以传入 pydantic 函数进行验证并且转换为 pydantic object
pydantic_obj = ResponseFormatter.model_validate(ai_msg.tool_calls[0]["args"])
print(pydantic_obj)
# 这就是原生的转换方法

我们也可以使用 LangChain 自带的 `JSON Mode` 方法（相比于前者并没有使用到 tools）。实际上这种模式的主要方法是强制模型输出合乎规范的 JSON（更像是不需要你声明的 Prompt Engineering）。能够使用 `JSON Mode` 的模型已经在[官方文档](https://python.langchain.com/docs/integrations/chat/#featured-providers)内列举了出来。使用方法非常的简单，只需要使用链式函数在后面加上 `.with_structured_output(method="json_mode")` 即可。

In [16]:
llm_with_structure = llm.with_structured_output(method="json_mode")
ai_msg = llm_with_structure.invoke(
    "Return a JSON object of identities of 2 random person with key 'name', 'gender', 'height', 'weight', 'eye color', 'hair color'."
)
print(type(ai_msg))
print(ai_msg)

<class 'dict'>
{'person1': {'name': 'Alex Johnson', 'gender': 'Male', 'height': '180 cm', 'weight': '75 kg', 'eye color': 'Brown', 'hair color': 'Black'}, 'person2': {'name': 'Emily Smith', 'gender': 'Female', 'height': '165 cm', 'weight': '60 kg', 'eye color': 'Blue', 'hair color': 'Blonde'}}


🤔 Andrew Ng 课程上使用的方法为 `ResponseSchema` 和 `StructuredOutputParser`，预先定义 JSON 中的 Key-Value 对，有点类似于 ORM 框架中的映射对象，可以在[这里](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/structured/)找到。但是当时的版本是 v0.1，目前文档已经标注了不再被维护，同时系统也会有 `Deprecated` 的警告。我们就不采用旧版本的写法，全部使用[新版本文档](https://python.langchain.com/docs/concepts/structured_outputs/#structured-output-method)中的写法。

操作精讲：之前的课程里，我们定义了三个 `ResponseSchema`，并使用 `StructuredOutputParser` 将其组织起来，从而获得了一个类似于 `Prompt Engineering` 的操作。具体的代码如下：
```python
gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else?")
attitude_schema = ResponseSchema(name="attitude", description="What is the attitude of this customer? Positive of Negative?")
price_value_schema = ResponseSchema(name="price_value", description="What is the cost-effectiveness of this product?")

output_parser = StructuredOutputParser.from_response_schemas([gift_schema, attitude_schema, price_value_schema])
print(output_parser.get_format_instructions())
```

我们打印出依次定义的 schema，可以发现就是在每个 instruction 的最后加上了人为定义的 Prompt，这也是 `with_structured_output` 的原理。只不过目前版本包装得非常多，把很多底层的 Prompt 都人为屏蔽掉了。以下就是每次输入的时候额外传入的 Prompt：

```text
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

\```json
{
	"gift": string  // Was the item purchased as a gift for someone else?
	"attitude": string  // What is the attitude of this customer? Positive of Negative?
	"price_value": string  // What is the cost-effectiveness of this product?
}
\```
```

OK，我们梳理一下目前的方法：
1. 一种是构建一个 pydantic 格式工具（名字类似于 `ResponseFormatter`），然后将 LLM 绑定这个工具 `llm.bind_tools([ResponseFormatter])`，最后使用 pydantic `ResponseFormatter.model_validate` 的格式转换方法变成我们需要的 JSON 格式；
2. 另一种是直接针对 LLM Provider，如果允许，使用自带的 `JSON Mode` 方法，我们 `invoke` 之后接收到的 `response` 本身就会成为 dict 类型的数据，约定于是一个 JSON，但是不一定所有的 Provider 都会适配一个 `JSON Mode`（比如 [DeepSeek](https://python.langchain.com/docs/integrations/chat/deepseek/) 就没有适配这个 mode）。

我们寻求一种比较好的模式，可以自己定义一个 pydantic 格式工具，然后将其作为参数传入 LLM 当中，并且返回来的参数直接就是 pydantic 格式，这样我们会少很多步骤。最新的官方文档中就提供了这一种模式 [Structure output method](https://python.langchain.com/docs/concepts/structured_outputs/#structured-output-method)，参考下图的流程：

![Bind Schema to LLM](https://python.langchain.com/assets/images/with_structured_output-4fd0fdc94f644554d52c6a8dee96ea21.png)

In [18]:
llm_with_structure = llm.with_structured_output(ResponseFormatter)
structured_out = llm_with_structure.invoke("Which is bigger? 9.11 or 9.9?")
print(structured_out)

answer='The number 9.9 is bigger than 9.11.\n\n### Explanation:\n- **9.11** can be broken down as 9 + 0.11.\n- **9.9** can be broken down as 9 + 0.9.\n\nWhen comparing the decimal parts:\n- 0.11 is less than 0.9.\n\nTherefore, 9.9 is greater than 9.11.' followup_question='Would you like to know how to compare other decimal numbers?'


In [30]:
print(type(structured_out))
# 将 ResponseFormatter 包装的回复强制转化成 JSON
print(ResponseFormatter.model_validate(structured_out).model_dump_json())

<class '__main__.ResponseFormatter'>
{"answer":"The number 9.9 is bigger than 9.11.\n\n### Explanation:\n- **9.11** can be broken down as 9 + 0.11.\n- **9.9** can be broken down as 9 + 0.9.\n\nWhen comparing the decimal parts:\n- 0.11 is less than 0.9.\n\nTherefore, 9.9 is greater than 9.11.","followup_question":"Would you like to know how to compare other decimal numbers?"}
